# **Quantitative Value Strategy**

In volatile and uncertain market times, intrinsic value investing takes on less of a risk than momentum investing, which is a stronger strategy in a bull market. 



"Value investing" means investing in the stocks that are the cheapest relative to common measures of business value (like earnings or assets).

For this project, we are going to build an investing strategy that selevts the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.